# ETL

In [74]:
import pandas as pd
import gzip
import ast
import json
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [108]:
dfgames = pd.read_json('datasets/steam_games.json.gz',compression='gzip',lines=True)
dfgames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [109]:

dfgames.dropna(how='all',inplace=True)
dfgames.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


In [110]:
dfgames[dfgames['id'].isnull()] # Buscamos filas con id nulos

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [111]:
dfgames.dropna(subset=['id'],inplace=True) # Eliminamos las filas

In [112]:
duplicados = dfgames['id'].duplicated(keep=False) # Buscamos duplicados por id
dfgames[duplicados]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


In [113]:
dfgames.drop_duplicates(subset='id',inplace=True ,keep='last') # Eliminamos los duplicados manteniendo el ultimo
dfgames.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24081 non-null  object 
 1   genres        28850 non-null  object 
 2   app_name      32131 non-null  object 
 3   title         30083 non-null  object 
 4   url           32132 non-null  object 
 5   release_date  30066 non-null  object 
 6   tags          31970 non-null  object 
 7   reviews_url   32132 non-null  object 
 8   specs         31463 non-null  object 
 9   price         30755 non-null  object 
 10  early_access  32132 non-null  float64
 11  id            32132 non-null  float64
 12  developer     28834 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


In [114]:
dfgames = dfgames.drop(['publisher', 'reviews_url','url', 'title', 'specs', 'early_access'], axis=1) # elimino las columnas que no necesito


In [115]:
dfgames.info()


<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 88310 to 120444
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28850 non-null  object 
 1   app_name      32131 non-null  object 
 2   release_date  30066 non-null  object 
 3   tags          31970 non-null  object 
 4   price         30755 non-null  object 
 5   id            32132 non-null  float64
 6   developer     28834 non-null  object 
dtypes: float64(1), object(6)
memory usage: 2.0+ MB


In [116]:


# Creando un conjunto de géneros únicos a partir de los datos de géneros no nulos en 'dfgames'
genres = set(item for val in dfgames['genres'].dropna() for item in val)

# Filtrando las etiquetas 'tags' para mantener solo aquellas que están presentes en 'genres'
dfgames['tags'] = dfgames['tags'].apply(lambda x: [item for item in x if item in genres] if isinstance(x, list) else x) 

# Rellenando valores nulos en la columna 'genres' con valores correspondientes de 'tags'
dfgames['genres'].fillna(dfgames['tags'], inplace=True)

def agregar_genres_tags(fila): # Función que añade valores de 'tags' a 'genres' que no estén ya presentes en 'genres'
    genres = fila['genres']
    tags = fila['tags']
    if isinstance(tags,list) and isinstance(genres,list):
        for tag in tags:
            if tag not in genres:
                genres.append(tag)
    return genres

# Aplicando la función 'agregar_genres_tags' a todos los elementos en el dataframe
dfgames['genres'] = dfgames.apply(lambda fila: agregar_genres_tags(fila), axis=1)

# Descartar la columna 'tags' ya que su información ya ha sido absorbida por 'genres'
dfgames.drop(columns=['tags'], inplace=True)

In [117]:
dfgames[dfgames['genres'].isna()]

,genres,app_name,release_date,price,id,developer
88668,None,Europa Universalis III: Heir to the Throne,2009-12-15,9.99,25806.0,Paradox Interactive
88779,None,Booster Trooper Demo,2010-08-31,None,27930.0,DnS Development
88922,None,"Warhammer 40,000: Dawn of War II - Retribution...",2011-02-28,0.99,56436.0,"Relic Entertainment,Feral Interactive (Mac/Linux)"
89089,None,Dungeon Defenders Halloween Costume Pack,2011-11-11,0.99,202520.0,Trendy Entertainment
89090,None,Dungeon Defenders Capture the Flag Pre-Alpha P...,2011-11-11,None,202523.0,Trendy Entertainment
...,...,...,...,...,...,...
119491,None,Total War: SHOGUN 2 - Sengoku Jidai Unit Pack,2011-07-28,2.99,34342.0,"The Creative Assembly,Feral Interactive (Mac),..."
119572,None,"Worms Reloaded: The ""Pre-order Forts and Hats""...",2011-05-17,1.99,22630.0,Team17 Digital Ltd
119583,None,Total War: SHOGUN 2 - The Ikko Ikki Clan Pack,2011-05-26,4.99,34348.0,"The Creative Assembly,Feral Interactive (Mac),..."
119634,None,"Killing Floor ""London's Finest"" Character Pack",2010-12-14,7.99,35419.0,Tripwire Interactive


In [118]:
# Convertir la lista de géneros a una cadena separada por puntos
dfgames['genres'] = dfgames['genres'].apply(lambda x: ".".join(x) if isinstance(x, list) else x)

# Crear variables ficticias para los géneros
dummies = dfgames['genres'].str.get_dummies(sep='.')

# Concatenar las variables ficticias con el DataFrame original
dfgames = pd.concat([dfgames, dummies], axis=1)

# Eliminar la columna 'genres'
dfgames.drop(columns='genres', inplace=True)

In [119]:
# Reemplazar los valores no válidos por NaN y convertir a tipo datetime
dfgames["release_date"] = pd.to_datetime(dfgames["release_date"], errors='coerce')

# Filtrar el DataFrame para obtener los registros válidos
df_filtered = dfgames.dropna(subset=["release_date"])

# Rellenar los valores faltantes en la columna 'release_date' con la moda
columns_fill_mode = ['release_date']
df_filtered[columns_fill_mode] = df_filtered[columns_fill_mode].fillna(df_filtered[columns_fill_mode].mode().iloc[0])

# Crear una nueva columna 'year' a partir de 'release_date'
df_filtered['year'] = df_filtered['release_date'].dt.year.astype('int')

# Eliminar la columna original 'release_date'
df_filtered.drop('release_date', axis=1, inplace=True)

# Asignar el DataFrame filtrado y procesado de nuevo a dfgames
dfgames = df_filtered.copy()


C:\Users\castr\AppData\Local\Temp\ipykernel_19768\2023665571.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[columns_fill_mode] = df_filtered[columns_fill_mode].fillna(df_filtered[columns_fill_mode].mode().iloc[0])
C:\Users\castr\AppData\Local\Temp\ipykernel_19768\2023665571.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['year'] = df_filtered['release_date'].dt.year.astype('int')
C:\Users\castr\AppData\Local\Temp\ipykernel_19768\2023665571.py:15: SettingWithCopyWarning: 
A 

In [120]:
dfgames

,app_name,price,id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
88310,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,2018
88311,Ironbound,Free To Play,643980.0,Secret Level SRL,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018
88312,Real Pool 3D - Poolians,Free to Play,670290.0,Poolians.com,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,2017
88313,弹炸人2222,0.99,767400.0,彼岸领域,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,2017
88315,Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Kebab it Up!,1.99,745400.0,Bidoniera Games,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,2018
120440,Colony On Mars,1.99,773640.0,"Nikita ""Ghost_RUS""",0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,2018
120441,LOGistICAL: South Africa,4.99,733530.0,Sacada,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,2018
120442,Russian Roads,1.99,610660.0,Laush Dmitriy Sergeevich,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,2018


In [151]:
dfgames.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27710 entries, 88310 to 120443
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   app_name                   27710 non-null  category
 1   price                      27710 non-null  float32 
 2   id                         27710 non-null  int16   
 3   developer                  27710 non-null  category
 4   Accounting                 27710 non-null  int32   
 5   Action                     27710 non-null  int32   
 6   Adventure                  27710 non-null  int32   
 7   Animation &amp; Modeling   27710 non-null  int32   
 8   Audio Production           27710 non-null  int32   
 9   Casual                     27710 non-null  int32   
 10  Design &amp; Illustration  27710 non-null  int32   
 11  Early Access               27710 non-null  int32   
 12  Education                  27710 non-null  int32   
 13  Free to Play               2771

In [135]:
dfgames['id'] = dfgames['id'].astype('int16')

In [123]:
generos = ['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']

# Cambiar el tipo de datos a int32
for genero in generos:
    dfgames[genero] = dfgames[genero].astype('int')

In [140]:
dfgames['app_name'] = dfgames['app_name'].astype('category')

In [137]:
for col in dfgames.columns:
    if dfgames[col].dtype == 'object':
        num_unique_values = len(dfgames[col].unique())
        num_total_values = len(dfgames[col])
        if num_unique_values / num_total_values < 0.5:
            dfgames[col] = dfgames[col].astype('category')

In [131]:
def limpiar_precio(precio):
    if isinstance(precio, str):
        if 'Free' in precio or 'Install Now' in precio:
            return 0.0
        elif 'Starting at $499.00' in precio:
            return 499.0
        elif 'Starting at $449.00' in precio:
            return 449.0
        else:
            return None
    else:
        return precio

# Aplica la función a la columna 'price'
dfgames['price'] = dfgames['price'].apply(limpiar_precio)

# Rellena los valores NaN en 'price' donde 'Free to Play' es 1 con 0.0
dfgames.loc[(dfgames['Free to Play'] == 1) & dfgames['price'].isna(), 'price'] = 0.0

# Elimina los valores NaN restantes en 'price'
dfgames = dfgames.dropna(subset=['price'])

# Convierte la columna 'price' a float
dfgames['price'] = dfgames['price'].astype('float32')

In [142]:
dfgames.isnull().sum()

app_name                     0
price                        0
id                           0
developer                    0
Accounting                   0
Action                       0
Adventure                    0
Animation &amp; Modeling     0
Audio Production             0
Casual                       0
Design &amp; Illustration    0
Early Access                 0
Education                    0
Free to Play                 0
Indie                        0
Massively Multiplayer        0
Photo Editing                0
RPG                          0
Racing                       0
Simulation                   0
Software Training            0
Sports                       0
Strategy                     0
Utilities                    0
Video Production             0
Web Publishing               0
year                         0
dtype: int64

In [127]:
dfgames.dropna(subset=['developer'],inplace=True)

In [145]:
dfgames.to_parquet('datasets/dfgames.parquet',index=False)

## USER_REVIEWS

In [41]:
user_reviews_gz = "datasets/user_reviews.json.gz"
data = []

with gzip.open(user_reviews_gz, 'rt', encoding='utf-8') as archivo:
    for line in archivo:
        data.append(ast.literal_eval(line))

df_review = pd.DataFrame(data)
df_review.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [42]:
# Crear un DataFrame con la columna 'reviews' desanidada
df_reviews_open = df_review.explode('reviews')

# Concatenar el DataFrame resultante con las columnas existentes
df_reviews_open = pd.concat([df_reviews_open.drop(['reviews'], axis=1), df_reviews_open['reviews'].apply(pd.Series)], axis=1)

In [43]:
df_reviews_open 

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D,NaN


In [44]:
df_reviews_open['posted'].isnull().sum()

28

In [45]:
df_reviews_open.dropna(subset=['posted'], inplace=True)

In [46]:
df_reviews_open['posted year'] = df_reviews_open['posted'].str.extract(r'(\d{4})', expand=False)


In [47]:
# Se elimina la columna posted
df_reviews_open.drop('posted' , axis = 1, inplace = True)

In [48]:
df_reviews_open.head()

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,0,posted year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,2011
1,js41637,http://steamcommunity.com/id/js41637,,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN,2014
1,js41637,http://steamcommunity.com/id/js41637,,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN,2013


In [49]:
df_reviews_open.drop(columns=['user_url','funny','helpful','last_edited',0],inplace=True)

In [50]:
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def analisis_sentimiento(review):
    if review and not pd.isnull(review):
        sentiment_score = sia.polarity_scores(review)
        if sentiment_score['compound'] >= 0.5:
            return 2  # Sentimiento positivo
        elif sentiment_score['compound'] <= -0.5:
            return 0  # Sentimiento negativo
        else:
            return 1  # Sentimiento neutral
    else:
        return 1

df_reviews_open['review'].fillna('',inplace=True) # Remplazo los nulos con un string vacio
# Aplicar la función a la columna 'review' y crear una nueva columna 'sentimiento'
df_reviews_open['sentiment_analysis'] = df_reviews_open['review'].apply(analisis_sentimiento)
df_reviews_open.drop(columns='review',inplace=True) # Elimino la columna review

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\castr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [51]:
df_reviews_open[df_reviews_open.isnull().any(axis=1)]

,user_id,item_id,recommend,posted year,sentiment_analysis
2,evcentric,248820,True,NaN,1
6,76561198079601835,730,True,NaN,1
7,MeaTCompany,730,True,NaN,1
9,76561198156664158,252950,True,NaN,2
10,76561198077246154,440,True,NaN,1
...,...,...,...,...,...
25797,76561198312638244,70,True,NaN,2
25797,76561198312638244,362890,True,NaN,2
25798,LydiaMorley,273110,True,NaN,2
25798,LydiaMorley,730,True,NaN,2


In [52]:
df_reviews_open.dropna(inplace=True)

In [53]:
df_reviews_open

,user_id,item_id,recommend,posted year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
0,76561197970982479,22200,True,2011,1
0,76561197970982479,43110,True,2011,2
1,js41637,251610,True,2014,2
1,js41637,227300,True,2013,2
...,...,...,...,...,...
25764,wayfeng,730,True,2015,1
25765,76561198251004808,253980,True,2015,2
25769,72947282842,730,True,2015,1
25771,ApxLGhost,730,True,2015,2


In [54]:
df_reviews_open.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49186 entries, 0 to 25780
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49186 non-null  object
 1   item_id             49186 non-null  object
 2   recommend           49186 non-null  object
 3   posted year         49186 non-null  object
 4   sentiment_analysis  49186 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.3+ MB


In [55]:
df_reviews_open['posted year'] = df_reviews_open['posted year'].astype('int')

In [56]:
df_reviews_open['item_id'] = df_reviews_open['item_id'].astype('int')

In [57]:
df_reviews_open['sentiment_analysis'] = df_reviews_open['sentiment_analysis'].astype('int')

In [58]:
for col in df_reviews_open.columns:
    if df_reviews_open[col].dtype == 'object':
        num_unique_values = len(df_reviews_open[col].unique())
        num_total_values = len(df_reviews_open[col])
        if num_unique_values / num_total_values < 0.5:
            df_reviews_open[col] = df_reviews_open[col].astype('category')

In [59]:
df_reviews_open.to_parquet('datasets/user_reviews.parquet',index=False)

## USERS_ITEM

In [61]:
def descomprimir_json(ruta, variable_anidada):

    filas = []

    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
      for line in archivo.readlines():
          filas.append(ast.literal_eval(line))

    df = pd.DataFrame(filas)                                                 
    df = df.explode(variable_anidada).reset_index()                         
    df = df.drop(columns="index")                                           
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)   
    df = df.drop(columns=variable_anidada)   

    return df

In [62]:
""" def descomprimir_json_optimizado(ruta, variable_anidada):
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
        for line in archivo:
            fila = ast.literal_eval(line)
            df = pd.json_normalize(fila, variable_anidada)
            yield df """

" def descomprimir_json_optimizado(ruta, variable_anidada):\n    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:\n        for line in archivo:\n            fila = ast.literal_eval(line)\n            df = pd.json_normalize(fila, variable_anidada)\n            yield df "

In [63]:
""" df_items = pd.concat(descomprimir_json_optimizado("datasets/users_items.json.gz", 'items')) """
df_items = descomprimir_json("datasets/users_items.json.gz", 'items')

In [64]:
df_items

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0


In [65]:
df_items_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype   
---  ------            -----   
 0   user_id           category
 1   item_id           int32   
 2   item_name         category
 3   playtime_forever  int32   
dtypes: category(2), int32(2)
memory usage: 111.0 MB


In [66]:
df_items_cl = df_items.drop(columns=['user_url','playtime_2weeks','steam_id','items_count'])

In [148]:
df_items_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype   
---  ------            -----   
 0   user_id           category
 1   item_id           int16   
 2   item_name         category
 3   playtime_forever  int16   
dtypes: category(2), int16(2)
memory usage: 91.4 MB


In [68]:
df_items_cl.isnull().sum()

user_id                 0
item_id             16806
item_name           16806
playtime_forever    16806
dtype: int64

In [69]:
df_items_cl.dropna(inplace=True)

In [146]:
df_items_cl['item_id'] = df_items_cl['item_id'].astype('int16')

In [147]:
df_items_cl['playtime_forever'] = df_items_cl['playtime_forever'].astype('int16')

In [72]:
for col in df_items_cl.columns:
    if df_items_cl[col].dtype == 'object':
        num_unique_values = len(df_items_cl[col].unique())
        num_total_values = len(df_items_cl[col])
        if num_unique_values / num_total_values < 0.5:
            df_items_cl[col] = df_items_cl[col].astype('category')

In [149]:
df_items_cl.to_parquet('datasets/users_item.parquet')

In [150]:
df_items_cl

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...
5170009,76561198329548331,18650,BrainBread 2,0
5170010,76561198329548331,-19886,All Is Dust,0
5170011,76561198329548331,-4726,One Way To Die: Steam Edition,3
5170012,76561198329548331,-2718,You Have 10 Seconds 2,4
